In [112]:
import pandas as pd
pd.set_option('max_colwidth', 100)
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import confusion_matrix
import itertools
import scipy.stats as st
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.feature_selection import mutual_info_classif
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)
import rpy2
#import pingouin as pg
from itertools import combinations

import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
import rpy2.ipython.html
rpy2.ipython.html.init_printing()
from rpy2.rinterface_lib.embedded import RRuntimeError

# import R's "utils" package
utils = importr('utils')
lavaan = importr('lavaan')
semtools = importr('semTools')
stringr = importr('stringr')
reticulate = importr('reticulate')

In [3]:
# importing stuff the right way
try:
    semTools_version = '.'.join([str(ii) for ii in list(ro.r('packageVersion("semTools")')[0])])
except RRuntimeError:
    semTools_version = ''
# There's a bug in the main semtools libarary, this install from my fork of semtools where I've fixed it.
if semTools_version != '0.5.6.933':
    ro.r("""
        library(devtools)
        install_github("shotgunosine/semTools/semTools@aa68e38f88d9dd881617d5ade429bde2b2a74e08")
        """
    )

In [74]:
def make_hitop_list(data):
    # only hitop cols
    hitop_cols = []
    for c in data.columns:
        if 'hitop' in c and 'today' not in c:
            hitop_cols.append(c)   
    return(hitop_cols)

def make_recontact_list(data):
    list_recontact = []
    for c in data.columns:
        if 'recontact' in c:
            list_recontact.append(c)
    return(list_recontact)

def do_checks(dat, remove_checks):
    dat['passed_checks'] = True
    dat['passed_grid'] = True
    dat['passed_list'] = True
    dat.loc[dat.check_moderately != 3, 'passed_checks'] = False
    dat.loc[dat.check_notatall != 1, 'passed_checks'] = False
    dat.loc[dat.check_moderately != 3, 'passed_grid'] = False
    dat.loc[dat.check_notatall != 1, 'passed_grid'] = False
    dat.loc[dat.todaycheck_1 != 1, 'passed_checks'] = False
    dat.loc[dat.todaycheck_2 != 1, 'passed_checks'] = False
    dat.loc[dat.todaycheck_1 != 1, 'passed_list'] = False
    dat.loc[dat.todaycheck_2 != 1, 'passed_list'] = False
    # deal with attention checks - recontact
    dat['passed_checks_recontact'] = True
    dat['passed_grid_recontact'] = True
    dat['passed_list_recontact'] = True
    dat.loc[dat.check_moderately_recontact != 3, 'passed_checks_recontact'] = False
    dat.loc[dat.check_notatall_recontact != 1, 'passed_checks_recontact'] = False
    dat.loc[dat.check_moderately_recontact != 3, 'passed_grid_recontact'] = False
    dat.loc[dat.check_notatall_recontact != 1, 'passed_grid_recontact'] = False
    dat.loc[dat.todaycheck_1_recontact != 1, 'passed_checks_recontact'] = False
    dat.loc[dat.todaycheck_2_recontact != 1, 'passed_checks_recontact'] = False
    dat.loc[dat.todaycheck_1_recontact != 1, 'passed_list_recontact'] = False
    dat.loc[dat.todaycheck_2_recontact != 1, 'passed_list_recontact'] = False
    print('checks_initial:')
    print(1 - dat.loc[:, ['passed_checks', 'passed_grid', 'passed_list']].mean())
    how_many_passed_checks = dat['passed_checks'].value_counts()[True]
    how_many_overall = dat.shape[0]
    print('Passed checks: %d out of %d' % (how_many_passed_checks, how_many_overall))
    print('checks_recontact:')
    print(1 - dat.loc[:, ['passed_checks_recontact', 'passed_grid_recontact', 'passed_list_recontact']].mean()) 
    how_many_passed_checks_recontact = dat['passed_checks_recontact'].value_counts()[True]
    print('Passed checks: %d out of %d' % (how_many_passed_checks_recontact, how_many_overall))
    if remove_checks:
        print('removing checks')
        dat = dat.loc[dat['passed_checks'] == True]
        dat = dat.loc[dat['passed_checks_recontact'] == True]
        print(dat.shape)
        print('done removing checks')
    return(dat)

def load_item_lookup():
    # this is a table that aligns scale items with their corresponding questions
    htqs = pd.read_excel('../data/ValSample/Internalizing-Somatoform Items_DW.xlsx', skiprows=2,
                                names=['id', 'scale', 'item', '_0', '_1', '_2', '_3']).drop(['_0', '_1', '_2', '_3'], axis=1)
    # cmqs is a dataframe with items and responses
    # htcm is the same dataframe, but only displaying HiTOP items
    cmqs = pd.read_csv('../data/CFA/cogmood_questions.csv')
    cmqs = cmqs.rename({'Unnamed: 0': 'id'}, axis=1)
    htcm = cmqs.query("measure == 'HiTOP'")
    # this is a lookup table for subscales, items (sentence questions), and hitop ids
    item_lookup = htcm.loc[:, ['id','subscale', 'item']].merge(htqs.loc[:, ['id','item']], how='inner', on='item', suffixes=['_cm', '_ht'])
    item_lookup['htid'] = 'hitop'+ item_lookup.id_ht.astype(str)
    return(item_lookup)
    
def load_valsample():
    # VALIDATION SAMPLE - main dataframe with responses
    dat_validation = pd.read_excel('../data/ValSample/inters23.xlsx')
    '''resp_cols = responses.columns[responses.columns.isin(item_lookup.htid)]
    # scale scores
    scale_scores = []
    for ss, df in item_lookup.groupby('subscale'):
        tmp = responses.loc[:, df.htid].sum(1)
        tmp.name = ss.replace(" ", "_").replace("/", "_").replace("-", "_")
        scale_scores.append(tmp)
    scale_scores = pd.concat(scale_scores, axis=1)
    scale_names = list(scale_scores.columns.values)'''
    #do_checks(dat_validation)    
    # this is the validation data to be used
    hitop_cols_list = make_hitop_list(dat_validation)
    data_validation = dat_validation[hitop_cols_list]
    data_validation = data_validation.assign(whichdata=['validation']*data_validation.shape[0])
    return(data_validation)

def do_codesheet():
    #loads the codesheet file
    # wherever the emane column exists, creates a dict variablename:ename
    code_book = pd.read_csv('../dylan_github/yougov_codesheet_alignment.tsv', sep='\t')
    ename_lut = {vn:en for vn, en in code_book.loc[code_book.ename.notnull(), ['var_name', 'ename']].values}
    for vn, en in code_book.loc[code_book.ename.notnull(), ['var_name', 'ename']].values:
        ename_lut[vn+'_recontact'] = en+'_recontact'
    tmp = code_book.query('ename.notnull()')
    scale_lut = {}
    for ss, df in tmp.loc[tmp.ename.str.contains('hitop') & ~tmp.ename.str.contains('today')].groupby('subscale'):
        scale_name = ss.replace(" ", "_").replace("/", "_").replace("-", "_")
        items = df.ename.values
        scale_lut[scale_name] = items    
     #   scale_name_initial = scale_name+'_initial'
     #   scale_lut[scale_name_initial] = items 
        scale_name_recontact = scale_name+'_recontact'
        items_recontact = []
        for item in items:
            items_recontact.append(item+'_recontact')
        scale_lut[scale_name_recontact] = np.array(items_recontact, dtype=object)
    # rename checks
    ename_lut['FNM_Q8_5'] ='check_moderately'
    ename_lut['FNM_Q22_3'] ='check_notatall'
    ename_lut['FNM_Q42_m_10'] ='todaycheck_1'
    ename_lut['FNM_44_m_28'] ='todaycheck_2'
    ename_lut['FNM_Q8_5_recontact'] ='check_moderately_recontact'
    ename_lut['FNM_Q22_3_recontact'] ='check_notatall_recontact'
    ename_lut['FNM_Q42_m_10_recontact'] ='todaycheck_1_recontact'
    ename_lut['FNM_44_m_28_recontact'] ='todaycheck_2_recontact'
    return ename_lut, scale_lut
    
def load_data(dataname, doing_checks, doing_remove_checks):
    print(doing_remove_checks)
    assert(dataname in ['genpop','enriched'])
    #codesheet file
    ename_lut, scale_lut = do_codesheet()
    # done working with the codesheet file, now opening the actual data
    if dataname == 'genpop':
        datapath = '../data/NIMH0007_genpop_num_OUTPUT.csv'
    elif dataname == 'enriched':
        datapath = '../data/NIMH0007_mental_health_num_OUTPUT.csv'
    dat = pd.read_csv(datapath, dtype={'caseid':str}, engine='python')
    #print(dat.head())
    if dataname == 'genpop':
        # ONLY GENPOP drop the .0 that pandas appends for some reason - only for genpop
        dat['caseid'] = dat.caseid.str[:-2]
    dat = dat.rename(ename_lut, axis=1)
    if doing_checks:
        dat = do_checks(dat = dat, remove_checks = doing_remove_checks) 
    hitop_cols_list = make_hitop_list(dat)
    data = dat[hitop_cols_list]
    data = data.assign(whichdata=[dataname]*data.shape[0])
    recontact_cols_list = make_recontact_list(data)
    data_norecontact = data.drop(columns=recontact_cols_list)
    return data, data_norecontact   

def prep_data_recontactanalysis(dataframe_withrecontact):
    df = dataframe_withrecontact
    df_helper = df.copy(deep=True)
    df_rows = df.shape[0]
    df = df.assign(whichvisit=['initialvisit']*df_rows)
    df_helper = df_helper.assign(whichvisit=['recontactvisit']*df_rows)
    # in df, drop all "recontact" columns
    # in df_helper, drop all initial visit columns, then rename the recontact columns
    cols_recontact = []
    cols_original = []
    for c in df.columns:
        if "recontact" in c:
            cols_recontact.append(c)
            cols_original.append(c[:-10])  
    df = df.drop(columns=cols_recontact)
    df_helper = df_helper.drop(columns=cols_original)
    cols_dict = {}
    for cname in cols_recontact:
        cols_dict[cname] = cname[:-10]
    # rename cols in df_helper
    df_helper.rename(columns=cols_dict, inplace=True)    
    mydata_withrecontact = pd.concat([df, df_helper])
    return(mydata_withrecontact)

def print_summary(obj):
    return rprint(summary(obj))

def print_short_summary(invariance_output):
    ro.r("myoutput <- capture.output(summary(invariance_output))") 
    ro.r("stringid <- grep(\"chisq\", myoutput)")
    ro.r("print(myoutput[stringid-1])")
    ro.r("print(myoutput[stringid])")
    return (0)

def print_problematic_items(invariance_output):
    ro.r("myoutput <- capture.output(summary(invariance_output))") 
    print('Problematic items:')
    ro.r("stringidsig <- grep(\"may differ between Groups\", myoutput)")
    ro.r("print(myoutput[stringidsig])")   
    return (0)
    
def extract_p(invariance_output):
    ro.globalenv['invariance_output'] = invariance_output
    ro.r("myoutput <- capture.output(summary(invariance_output))") 
    ro.r("stringid <- grep(\"chisq\", myoutput)")
    ro.r('myrline <- myoutput[stringid]')
    mypline = ro.r('myrline')[0]
    my_p = mypline.split()[-1]
    return(my_p)

def check_secondary_criteria(fit_config):
    Criteria_passed = False
    ro.r("myoutputconfig <- capture.output(summary(fit_config, fit.measures=TRUE))")
    ro.r("stringid_cfi <- grep(\"Robust.*CFI\", myoutputconfig)")
    ro.r("cfi_line <- myoutputconfig[stringid_cfi]")
    cfiline = ro.r('cfi_line')[0]    
    my_cfi = cfiline.split()[-1] 
    my_cfi = float(my_cfi)
    ro.r("stringid_tli <- grep(\"Robust.*TLI\", myoutputconfig)")
    ro.r("tli_line <- myoutputconfig[stringid_tli]")
    tliline = ro.r('tli_line')[0]    
    my_tli = tliline.split()[-1] 
    my_tli = float(my_tli)
    ro.r("stringid_rmsea <- grep(\"Robust.*RMSEA\", myoutputconfig)")
    ro.r("rmsea_line <- myoutputconfig[stringid_rmsea]")
    myrmsealine = ro.r('rmsea_line')[0]
    my_rmsea = myrmsealine.split('\n')[0].split()[-1]
    my_rmsea = float(my_rmsea)   
    if (my_cfi > 0.95) and (my_tli > 0.95) and (my_rmsea < 0.06):
        Criteria_passed = True
    return (Criteria_passed, my_cfi, my_tli, my_rmsea)
    
def do_cfa(mydata, whichcolumn):
    assert(whichcolumn in ['valid_enriched', 'valid_genpop', 'enriched_withrecontact', 'genpop_withrecontact'])
    num_iter = 1000
    rprint = ro.r('print')
    summary = ro.r('summary')
    ro.r('set.seed(12345)')
    rngkind = "L'Ecuyer-CMRG"
    mdl_strs = {}
    for ss, df in item_lookup.groupby('subscale'):
        cln_ss = ss.replace(" ", "_").replace("/", "_").replace("-", "_")
        items = []
        for itemn, item in enumerate(df.sort_values('id_ht').htid.values):
            items.append(item)
        mdl_strs[cln_ss] = f'{cln_ss} =~' + ' + '.join(items) 
    # measures we can't work with because too few degrees of freedom:
    # - 'appetite_loss': 'appetite_loss =~hitop109 + hitop280 + hitop283'
    # - 'cognitive_problems': 'cognitive_problems =~hitop142 + hitop159 + hitop189',
    # - 'indecisiveness': 'indecisiveness =~hitop21 + hitop90 + hitop95'
    # here's the dictionary with items that *will* work
    mdl_strs_new = {'anhedonic_depression': 'anhedonic_depression =~hitop39 + hitop77 + hitop84 + hitop92 + hitop93 + hitop123 + hitop157 + hitop182 + hitop230 + hitop246',
     'anxious_worry': 'anxious_worry =~hitop20 + hitop34 + hitop89 + hitop203 + hitop248 + hitop265',
     'appetite_gain': 'appetite_gain =~hitop120 + hitop141 + hitop243 + hitop275',
     'hyposomnia': 'hyposomnia =~hitop5 + hitop66 + hitop99 + hitop181 + hitop231',
     'insomnia': 'insomnia =~hitop160 + hitop254 + hitop261 + hitop268',
     'panic': 'panic =~hitop15 + hitop104 + hitop126 + hitop211 + hitop215 + hitop257',
     'separation_insecurity': 'separation_insecurity =~hitop40 + hitop50 + hitop69 + hitop81 + hitop113 + hitop136 + hitop151 + hitop197',
     'shame_guilt': 'shame_guilt =~hitop72 + hitop140 + hitop143 + hitop220',
     'situational_phobia': 'situational_phobia =~hitop16 + hitop165 + hitop225 + hitop247 + hitop278',
     'social_anxiety': 'social_anxiety =~hitop1 + hitop17 + hitop114 + hitop117 + hitop124 + hitop129 + hitop204 + hitop222 + hitop236 + hitop258',
     'well_being': 'well_being =~hitop9 + hitop23 + hitop54 + hitop106 + hitop149 + hitop200 + hitop244 + hitop245 + hitop250 + hitop281'}
    # to load the data in R, let's save it as csv and then load it to R
    # there may be a more elegant day to do this, but I don't know it...
    if whichcolumn == 'valid_genpop':
        mydata.to_csv('../data/CFA/mydata_valid_genpop.csv')
        ro.r("rdata <- read.csv('../data/CFA/mydata_valid_genpop.csv', header=TRUE)")
        group = 'whichdata'
    elif whichcolumn == 'valid_enriched':
        mydata.to_csv('../data/CFA/mydata_valid_enriched.csv')
        ro.r("rdata <- read.csv('../data/CFA/mydata_valid_enriched.csv', header=TRUE)") 
        group = 'whichdata'
    elif whichcolumn == 'enriched_withrecontact':
        mydata.to_csv('../data/CFA/data_enriched.csv')
        ro.r("rdata <- read.csv('../data/CFA/data_enriched.csv', header=TRUE)") 
        group = 'whichvisit'
    elif whichcolumn == 'genpop_withrecontact':
        mydata.to_csv('../data/CFA/data_genpop.csv')
        ro.r("rdata <- read.csv('../data/CFA/data_genpop.csv', header=TRUE)")
        group = 'whichvisit'
    ro.globalenv['group'] = group
    # CFA
    df_data = []    
    for cln_ss, mdl in mdl_strs_new.items():
        do_metric = False
        do_scalar = False
        do_strict = False
        print("\n\n\n================================")
        print("================================")
        print(cln_ss.upper())
        print(mdl)
        print("================================")
        print("================================")
        # ====== CONFIG =====        
        ro.globalenv[cln_ss+'_mdl'] = mdl_strs_new[cln_ss]
        fit_config = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV")')
        ro.globalenv['fit_config'] = fit_config        
        print("----------------")
        out_config = semtools.permuteMeasEq(nPermute=num_iter, # 100 for now, 1000 for the "real" run
                                                con=fit_config, # In the case of testing configural invariance when modelType = "mgcfa", con is the configural model (implicitly, the unconstrained model is the saturated model, so use the defaults uncon = NULL and param = NULL).
                                                parallelType="multicore", ncpus=4)
        config_p = extract_p(out_config)
        if float(config_p) >= 0.05:
            print('CONFIG INVARIANT chisq p = ' + str(config_p))
            do_metric = True
        else:
            (passed_flag, my_cfi, my_tli, my_rmsea) = check_secondary_criteria(fit_config)
            if passed_flag:
                print('CONFIG chisq p = ' + str(config_p))
                print('PASSED SECONDARY CRITERIA WITH CFI = ' + str(my_cfi) + ' TLI = ' + str(my_tli) + ' RMSEA = ' + str(my_rmsea))
                do_metric = True
            else:
                print('CONFIG INVARIANT NOT PASSED, chisq p = ' + str(config_p))
       
        if do_metric:
            # ====== METRIC =====
            fit_metric = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal="loadings")')
            ro.globalenv['fit_metric'] = fit_metric
            out_metric = semtools.permuteMeasEq(nPermute=num_iter, 
                                                uncon=fit_config, 
                                                con=fit_metric, 
                                                param="loadings",
                                                parallelType="multicore", ncpus=4)
            metric_p = extract_p(out_metric)
            if float(metric_p) >= 0.05:
                print('METRIC INVARIANT chisq p = ' + str(metric_p))
                do_scalar = True
            else:
                # significant:
                print('METRIC INVARIANT NOT PASSED, chisq p = ' + str(metric_p))
                _ = print_problematic_items(out_metric)
            
            if do_scalar:
                # ====== SCALAR =====
                ro.globalenv['out_metric'] = out_metric
                fit_scalar = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal=c("loadings", "intercepts"))')
                ro.globalenv['fit_scalar'] = fit_scalar
                out_scalar = semtools.permuteMeasEq(nPermute=num_iter, 
                                                uncon=fit_metric, 
                                                con=fit_scalar, 
                                                param=ro.StrVector(["loadings","intercepts"]),
                                                parallelType="multicore", 
                                                ncpus=4)
                scalar_p = extract_p(out_scalar)
                if float(scalar_p) >= 0.05:
                    print('SCALAR INVARIANT chisq p = ' + str(scalar_p))
                    do_strict = True
                else:
                    # significant:
                    print('SCALAR INVARIANT NOT PASSED, chisq p = ' + str(scalar_p))
                    _ = print_problematic_items(out_scalar)

                if do_strict:
                    # ====== STRICT =====
                    ro.globalenv['out_scalar'] = out_scalar
                    fit_strict = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal=c("loadings", "intercepts", "residuals"))')
                    ro.globalenv['fit_strict'] = fit_strict
                    out_strict = semtools.permuteMeasEq(nPermute=num_iter, 
                                                        uncon=fit_scalar, 
                                                        con=fit_strict, 
                                                        param=ro.StrVector(["loadings","intercepts", "residuals"]),
                                                        parallelType="multicore", 
                                                        ncpus=4)
                    strict_p = extract_p(out_strict)
                    if float(strict_p) >= 0.05:
                        print('STRICT INVARIANT chisq p = ' + str(strict_p))
                    else:
                        # significant:
                        print('STRICT INVARIANT NOT PASSED, chisq p = ' + str(strict_p))
                        _ = print_problematic_items(out_strict)

                else:
                    print('STRICT N/A')
                    strict_p = 'NA'
            else:
                print('SCALAR N/A')
                print('STRICT N/A')
                scalar_p = 'NA'
                strict_p = 'NA'
        else:
            print('METRIC N/A')
            print('SCALAR N/A')
            print('STRICT N/A')
            metric_p = 'NA'
            scalar_p = 'NA'
            strict_p = 'NA'
        df_data.append({cln_ss: [config_p, metric_p, scalar_p, strict_p]})
    df = pd.DataFrame(df_data)
    print("DONE")
    return(df)

def run_specific_cfa(mydata, whichcolumn, whichscale, whichcfa):
    print('! DONT FORGET TO CHANGE MDL_STRS_NEW BELOW IN CODE FOR ABLATION EXPERIMENTS')
    assert(whichcolumn in ['valid_enriched', 'valid_genpop', 'enriched_withrecontact', 'genpop_withrecontact'])
    assert(whichscale in ['anhedonic_depression', 'anxious_worry', 'appetite_gain', 'hyposomnia', 'insomnia', 'panic', 'separation_insecurity',
                         'shame_guilt', 'situational_phobia', 'social_anxiety', 'well_being'])
    num_iter = 1000
    rprint = ro.r('print')
    summary = ro.r('summary')
    ro.r('set.seed(12345)')
    rngkind = "L'Ecuyer-CMRG"
    mdl_strs = {}
    for ss, df in item_lookup.groupby('subscale'):
        cln_ss = ss.replace(" ", "_").replace("/", "_").replace("-", "_")
        items = []
        for itemn, item in enumerate(df.sort_values('id_ht').htid.values):
            items.append(item)
        mdl_strs[cln_ss] = f'{cln_ss} =~' + ' + '.join(items) 
    # measures we can't work with because too few degrees of freedom:
    # - 'appetite_loss': 'appetite_loss =~hitop109 + hitop280 + hitop283'
    # - 'cognitive_problems': 'cognitive_problems =~hitop142 + hitop159 + hitop189',
    # - 'indecisiveness': 'indecisiveness =~hitop21 + hitop90 + hitop95'
    # here's the dictionary with items that *will* work
    mdl_strs_new = {'anhedonic_depression': 'anhedonic_depression =~hitop39 + hitop77 + hitop84 + hitop92 + hitop93 + hitop123 + hitop157 + hitop182 + hitop230 + hitop246',
     'anxious_worry': 'anxious_worry =~hitop20 + hitop34 + hitop89 + hitop203 + hitop248 + hitop265',
     'appetite_gain': 'appetite_gain =~hitop120 + hitop141 + hitop243 + hitop275',
     'hyposomnia': 'hyposomnia =~hitop99 + hitop181 + hitop5 + hitop66 + hitop231',
     'insomnia': 'insomnia =~hitop160 + hitop254 + hitop261 + hitop268',
     'panic': 'panic =~hitop15 + hitop104 + hitop126 + hitop211 + hitop215 + hitop257',
     'separation_insecurity': 'separation_insecurity =~hitop40 + hitop50 + hitop69 + hitop81 + hitop113 + hitop136 + hitop151 + hitop197',
     'shame_guilt': 'shame_guilt =~hitop72 + hitop140 + hitop143 + hitop220',
     'situational_phobia': 'situational_phobia =~hitop16 + hitop165 + hitop225 + hitop247 + hitop278',
     'social_anxiety': 'social_anxiety =~hitop1 + hitop17 + hitop114 + hitop117 + hitop124 + hitop129 + hitop204 + hitop222 + hitop236 + hitop258',
     'well_being': 'well_being =~hitop9 + hitop23 + hitop54 + hitop106 + hitop149 + hitop200 + hitop244 + hitop245 + hitop250 + hitop281'}
    # to load the data in R, let's save it as csv and then load it to R
    # there may be a more elegant day to do this, but I don't know it...
    if whichcolumn == 'valid_genpop':
        mydata.to_csv('../data/CFA/mydata_valid_genpop.csv')
        ro.r("rdata <- read.csv('../data/CFA/mydata_valid_genpop.csv', header=TRUE)")
        group = 'whichdata'
    elif whichcolumn == 'valid_enriched':
        mydata.to_csv('../data/CFA/mydata_valid_enriched.csv')
        ro.r("rdata <- read.csv('../data/CFA/mydata_valid_enriched.csv', header=TRUE)") 
        group = 'whichdata'
    elif whichcolumn == 'enriched_withrecontact':
        mydata.to_csv('../data/CFA/data_enriched.csv')
        ro.r("rdata <- read.csv('../data/CFA/data_enriched.csv', header=TRUE)") 
        group = 'whichvisit'
    elif whichcolumn == 'genpop_withrecontact':
        mydata.to_csv('../data/CFA/data_genpop.csv')
        ro.r("rdata <- read.csv('../data/CFA/data_genpop.csv', header=TRUE)")
        group = 'whichvisit'
    ro.globalenv['group'] = group
    for cln_ss, mdl in mdl_strs_new.items():
        if cln_ss == whichscale:  
            print('lets go')
            if whichcfa == 'metric':
                do_metric = True
                do_scalar = False
                do_strict = False
            elif whichcfa == 'scalar':
                do_metric = True
                do_scalar = True
                do_strict = False   
            elif whichcfa == 'strict':
                do_metric = True
                do_scalar = True
                do_strict = True                  
            print("\n")
            print(cln_ss.upper())
            print(mdl)
            # ====== CONFIG =====        
            ro.globalenv[cln_ss+'_mdl'] = mdl_strs_new[cln_ss]
            fit_config = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV")')
            ro.globalenv['fit_config'] = fit_config        
            out_config = semtools.permuteMeasEq(nPermute=num_iter, # 100 for now, 1000 for the "real" run
                                                    con=fit_config, # In the case of testing configural invariance when modelType = "mgcfa", con is the configural model (implicitly, the unconstrained model is the saturated model, so use the defaults uncon = NULL and param = NULL).
                                                    parallelType="multicore", ncpus=4)
            config_p = extract_p(out_config)
            if float(config_p) >= 0.05:
                print('CONFIG INVARIANT chisq p = ' + str(config_p))
            else:
                (passed_flag, my_cfi, my_tli, my_rmsea) = check_secondary_criteria(fit_config)
                if passed_flag:
                    print('CONFIG chisq p = ' + str(config_p))
                    print('PASSED SECONDARY CRITERIA WITH CFI = ' + str(my_cfi) + ' TLI = ' + str(my_tli) + ' RMSEA = ' + str(my_rmsea))
                else:
                    print('CONFIG INVARIANT NOT PASSED, chisq p = ' + str(config_p))
            if do_metric:
                # ====== METRIC =====
                fit_metric = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal="loadings")')
                ro.globalenv['fit_metric'] = fit_metric
                out_metric = semtools.permuteMeasEq(nPermute=num_iter, 
                                                    uncon=fit_config, 
                                                    con=fit_metric, 
                                                    param="loadings",
                                                    parallelType="multicore", ncpus=4)
                metric_p = extract_p(out_metric)
                if float(metric_p) >= 0.05:
                    print('METRIC INVARIANT chisq p = ' + str(metric_p))
                else:
                    # significant:
                    print('METRIC INVARIANT NOT PASSED, chisq p = ' + str(metric_p))
                    ro.globalenv['out_metric'] = out_metric
                    ro.r("myoutputmetric <- capture.output(summary(out_metric))") 
                    ro.r("stringid <- grep(\"chisq\", myoutputmetric)")
                    ro.r("print(myoutputmetric[stringid-1])")
                    ro.r("print(myoutputmetric[stringid])")
                    # significant:
                    ro.r("stringidsig <- grep(\"may differ between Groups\", myoutputmetric)")
                    ro.r("print(myoutputmetric[stringidsig])")
                if do_scalar:
                    # ====== SCALAR =====
                    ro.globalenv['out_metric'] = out_metric
                    fit_scalar = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal=c("loadings", "intercepts"))')
                    ro.globalenv['fit_scalar'] = fit_scalar
                    out_scalar = semtools.permuteMeasEq(nPermute=num_iter, 
                                                    uncon=fit_metric, 
                                                    con=fit_scalar, 
                                                    param=ro.StrVector(["loadings","intercepts"]),
                                                    parallelType="multicore", 
                                                    ncpus=4)
                    scalar_p = extract_p(out_scalar)
                    if float(scalar_p) >= 0.05:
                        print('SCALAR INVARIANT chisq p = ' + str(scalar_p))
                    else:
                        # significant:
                        print('SCALAR INVARIANT NOT PASSED, chisq p = ' + str(scalar_p))
                        _ = print_problematic_items(out_scalar)
                        ro.globalenv['out_scalar'] = out_scalar
                        ro.r("myoutputscalar <- capture.output(summary(out_scalar))") 
                        ro.r("stringid <- grep(\"chisq\", myoutputscalar)")
                        ro.r("print(myoutputscalar[stringid-1])")
                        ro.r("print(myoutputscalar[stringid])")
                        # significant:
                        ro.r("stringidsig <- grep(\"may differ between Groups\", myoutputscalar)")
                        ro.r("print(myoutputscalar[stringidsig])")
                    if do_strict:
                        # ====== STRICT =====
                        ro.globalenv['out_scalar'] = out_scalar
                        fit_strict = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal=c("loadings", "intercepts", "residuals"))')
                        ro.globalenv['fit_strict'] = fit_strict
                        out_strict = semtools.permuteMeasEq(nPermute=num_iter, 
                                                            uncon=fit_scalar, 
                                                            con=fit_strict, 
                                                            param=ro.StrVector(["loadings","intercepts", "residuals"]),
                                                            parallelType="multicore", 
                                                            ncpus=4)
                        strict_p = extract_p(out_strict)
                        if float(strict_p) >= 0.05:
                            print('STRICT INVARIANT chisq p = ' + str(strict_p))
                        else:
                            # significant:
                            print('STRICT INVARIANT NOT PASSED, chisq p = ' + str(strict_p))
                            _ = print_problematic_items(out_strict)
                            ro.globalenv['out_strict'] = out_strict
                            ro.r("myoutputstrict <- capture.output(summary(out_strict))") 
                            ro.r("stringid <- grep(\"chisq\", myoutputstrict)")
                            ro.r("print(myoutputstrict[stringid-1])")
                            ro.r("print(myoutputstrict[stringid])")
                            # significant:
                            ro.r("stringidsig <- grep(\"may differ between Groups\", myoutputstrict)")
                            ro.r("print(myoutputstrict[stringidsig])")
                    else:
                        print('STRICT N/A')
                        strict_p = 'NA'
                else:
                    print('SCALAR N/A')
                    print('STRICT N/A')
                    scalar_p = 'NA'
                    strict_p = 'NA'
            else:
                print('METRIC N/A')
                print('SCALAR N/A')
                print('STRICT N/A')
                metric_p = 'NA'
                scalar_p = 'NA'
                strict_p = 'NA'
    print("DONE")
    return(0)    
    

In [109]:
def run_specific_cfa2(mydata, whichcolumn, whichscale, whichcfa):
    print('! DONT FORGET TO CHANGE MDL_STRS_NEW BELOW IN CODE FOR ABLATION EXPERIMENTS')
    assert(whichcolumn in ['valid_enriched', 'valid_genpop', 'enriched_withrecontact', 'genpop_withrecontact'])
    assert(whichscale in ['anhedonic_depression', 'anxious_worry', 'appetite_gain', 'hyposomnia', 'insomnia', 'panic', 'separation_insecurity',
                         'shame_guilt', 'situational_phobia', 'social_anxiety', 'well_being'])
    num_iter = 1000
    rprint = ro.r('print')
    summary = ro.r('summary')
    ro.r('set.seed(12345)')
    rngkind = "L'Ecuyer-CMRG"
    mdl_strs = {}
    for ss, df in item_lookup.groupby('subscale'):
        cln_ss = ss.replace(" ", "_").replace("/", "_").replace("-", "_")
        items = []
        for itemn, item in enumerate(df.sort_values('id_ht').htid.values):
            items.append(item)
        mdl_strs[cln_ss] = f'{cln_ss} =~' + ' + '.join(items) 
    # measures we can't work with because too few degrees of freedom:
    # - 'appetite_loss': 'appetite_loss =~hitop109 + hitop280 + hitop283'
    # - 'cognitive_problems': 'cognitive_problems =~hitop142 + hitop159 + hitop189',
    # - 'indecisiveness': 'indecisiveness =~hitop21 + hitop90 + hitop95'
    # here's the dictionary with items that *will* work
    mdl_strs_new = {'anhedonic_depression': 'anhedonic_depression =~hitop39 + hitop77 + hitop84 + hitop92 + hitop93 + hitop123 + hitop157 + hitop182 + hitop230 + hitop246',
     'anxious_worry': 'anxious_worry =~hitop20 + hitop34 + hitop89 + hitop203 + hitop248 + hitop265',
     'appetite_gain': 'appetite_gain =~hitop120 + hitop141 + hitop243 + hitop275',
     'hyposomnia': 'hyposomnia =~hitop99 + hitop181 + hitop5 + hitop66 + hitop231',
     'insomnia': 'insomnia =~hitop160 + hitop254 + hitop261 + hitop268',
     'panic': 'panic =~hitop15 + hitop104 + hitop126 + hitop211 + hitop215 + hitop257',
     'separation_insecurity': 'separation_insecurity =~hitop40 + hitop50 + hitop69 + hitop81 + hitop113 + hitop136 + hitop151 + hitop197',
     'shame_guilt': 'shame_guilt =~hitop72 + hitop140 + hitop143 + hitop220',
     'situational_phobia': 'situational_phobia =~hitop16 + hitop165 + hitop225 + hitop247 + hitop278',
     'social_anxiety': 'social_anxiety =~hitop1 + hitop17 + hitop114 + hitop117 + hitop124 + hitop129 + hitop204 + hitop222 + hitop236 + hitop258',
     'well_being': 'well_being =~hitop9 + hitop23 + hitop54 + hitop106 + hitop149 + hitop200 + hitop244 + hitop245 + hitop250 + hitop281'}
    # to load the data in R, let's save it as csv and then load it to R
    # there may be a more elegant day to do this, but I don't know it...
    if whichcolumn == 'valid_genpop':
        mydata.to_csv('../data/CFA/mydata_valid_genpop.csv')
        ro.r("rdata <- read.csv('../data/CFA/mydata_valid_genpop.csv', header=TRUE)")
        group = 'whichdata'
    elif whichcolumn == 'valid_enriched':
        mydata.to_csv('../data/CFA/mydata_valid_enriched.csv')
        ro.r("rdata <- read.csv('../data/CFA/mydata_valid_enriched.csv', header=TRUE)") 
        group = 'whichdata'
    elif whichcolumn == 'enriched_withrecontact':
        mydata.to_csv('../data/CFA/data_enriched.csv')
        ro.r("rdata <- read.csv('../data/CFA/data_enriched.csv', header=TRUE)") 
        group = 'whichvisit'
    elif whichcolumn == 'genpop_withrecontact':
        mydata.to_csv('../data/CFA/data_genpop.csv')
        ro.r("rdata <- read.csv('../data/CFA/data_genpop.csv', header=TRUE)")
        group = 'whichvisit'
    ro.globalenv['group'] = group
    for cln_ss, mdl in mdl_strs_new.items():
        if cln_ss == whichscale:  
            print('lets go')
            if whichcfa == 'metric':
                do_metric = True
                do_scalar = False
                do_strict = False
            elif whichcfa == 'scalar':
                do_metric = True
                do_scalar = True
                do_strict = False   
            elif whichcfa == 'strict':
                do_metric = True
                do_scalar = True
                do_strict = True                  
            print("\n")
            print(cln_ss.upper())
            print(mdl)
            # ====== CONFIG =====        

            # loop to do ablations
            temp_items = mdl_strs_new[cln_ss]
            temp_items_items = temp_items.split("=~",1)[1]
            temp_items_items_items = temp_items_items.split(" + ")
            list_of_items = temp_items_items_items
    
            for item in list_of_items:
                print('\nITEM TO REMOVE: ' + item)
                mynewstring = temp_items.split("=~",1)[0] + "=~"
                temp_items_items_items.remove(item)
                for x in temp_items_items_items:
                    mynewstring += x + ' + '
                temp_items_items_items = temp_items_items.split(" + ")
                mynewstring = mynewstring[:-3]
                print(mynewstring)
            
                ro.globalenv[cln_ss+'_mdl'] = mynewstring           
            
                fit_config = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV")')
                ro.globalenv['fit_config'] = fit_config        
                out_config = semtools.permuteMeasEq(nPermute=num_iter, # 100 for now, 1000 for the "real" run
                                                        con=fit_config, # In the case of testing configural invariance when modelType = "mgcfa", con is the configural model (implicitly, the unconstrained model is the saturated model, so use the defaults uncon = NULL and param = NULL).
                                                        parallelType="multicore", ncpus=4)
                config_p = extract_p(out_config)
                if float(config_p) >= 0.05:
                    print('CONFIG INVARIANT chisq p = ' + str(config_p))
                else:
                    (passed_flag, my_cfi, my_tli, my_rmsea) = check_secondary_criteria(fit_config)
                    if passed_flag:
                        print('CONFIG chisq p = ' + str(config_p))
                        print('PASSED SECONDARY CRITERIA WITH CFI = ' + str(my_cfi) + ' TLI = ' + str(my_tli) + ' RMSEA = ' + str(my_rmsea))
                    else:
                        print('CONFIG INVARIANT NOT PASSED, chisq p = ' + str(config_p))
                if do_metric:
                    # ====== METRIC =====
                    fit_metric = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal="loadings")')
                    ro.globalenv['fit_metric'] = fit_metric
                    out_metric = semtools.permuteMeasEq(nPermute=num_iter, 
                                                        uncon=fit_config, 
                                                        con=fit_metric, 
                                                        param="loadings",
                                                        parallelType="multicore", ncpus=4)
                    metric_p = extract_p(out_metric)
                    if float(metric_p) >= 0.05:
                        print('METRIC INVARIANT chisq p = ' + str(metric_p))
                    else:
                        # significant:
                        print('METRIC INVARIANT NOT PASSED, chisq p = ' + str(metric_p))
                        ro.globalenv['out_metric'] = out_metric
                        ro.r("myoutputmetric <- capture.output(summary(out_metric))") 
                        ro.r("stringid <- grep(\"chisq\", myoutputmetric)")
                        ro.r("print(myoutputmetric[stringid-1])")
                        ro.r("print(myoutputmetric[stringid])")
                        # significant:
                        ro.r("stringidsig <- grep(\"may differ between Groups\", myoutputmetric)")
                        ro.r("print(myoutputmetric[stringidsig])")
                    if do_scalar:
                        # ====== SCALAR =====
                        ro.globalenv['out_metric'] = out_metric
                        fit_scalar = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal=c("loadings", "intercepts"))')
                        ro.globalenv['fit_scalar'] = fit_scalar
                        out_scalar = semtools.permuteMeasEq(nPermute=num_iter, 
                                                        uncon=fit_metric, 
                                                        con=fit_scalar, 
                                                        param=ro.StrVector(["loadings","intercepts"]),
                                                        parallelType="multicore", 
                                                        ncpus=4)
                        scalar_p = extract_p(out_scalar)
                        if float(scalar_p) >= 0.05:
                            print('SCALAR INVARIANT chisq p = ' + str(scalar_p))
                        else:
                            # significant:
                            print('SCALAR INVARIANT NOT PASSED, chisq p = ' + str(scalar_p))
                            _ = print_problematic_items(out_scalar)
                            ro.globalenv['out_scalar'] = out_scalar
                            ro.r("myoutputscalar <- capture.output(summary(out_scalar))") 
                            ro.r("stringid <- grep(\"chisq\", myoutputscalar)")
                            ro.r("print(myoutputscalar[stringid-1])")
                            ro.r("print(myoutputscalar[stringid])")
                            # significant:
                            ro.r("stringidsig <- grep(\"may differ between Groups\", myoutputscalar)")
                            ro.r("print(myoutputscalar[stringidsig])")
                        if do_strict:
                            # ====== STRICT =====
                            ro.globalenv['out_scalar'] = out_scalar
                            fit_strict = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal=c("loadings", "intercepts", "residuals"))')
                            ro.globalenv['fit_strict'] = fit_strict
                            out_strict = semtools.permuteMeasEq(nPermute=num_iter, 
                                                                uncon=fit_scalar, 
                                                                con=fit_strict, 
                                                                param=ro.StrVector(["loadings","intercepts", "residuals"]),
                                                                parallelType="multicore", 
                                                                ncpus=4)
                            strict_p = extract_p(out_strict)
                            if float(strict_p) >= 0.05:
                                print('STRICT INVARIANT chisq p = ' + str(strict_p))
                            else:
                                # significant:
                                print('STRICT INVARIANT NOT PASSED, chisq p = ' + str(strict_p))
                                _ = print_problematic_items(out_strict)
                                ro.globalenv['out_strict'] = out_strict
                                ro.r("myoutputstrict <- capture.output(summary(out_strict))") 
                                ro.r("stringid <- grep(\"chisq\", myoutputstrict)")
                                ro.r("print(myoutputstrict[stringid-1])")
                                ro.r("print(myoutputstrict[stringid])")
                                # significant:
                                ro.r("stringidsig <- grep(\"may differ between Groups\", myoutputstrict)")
                                ro.r("print(myoutputstrict[stringidsig])")
                        else:
                            print('STRICT N/A')
                            strict_p = 'NA'
                    else:
                        print('SCALAR N/A')
                        print('STRICT N/A')
                        scalar_p = 'NA'
                        strict_p = 'NA'
                else:
                    print('METRIC N/A')
                    print('SCALAR N/A')
                    print('STRICT N/A')
                    metric_p = 'NA'
                    scalar_p = 'NA'
                    strict_p = 'NA'
    print("DONE")
    return(0)

In [114]:
mylist = ['a', 'b', 'c', 'd', 'e', 'f', 'g']
for com in combinations(mylist, 4):
    print(com)

('a', 'b', 'c', 'd')
('a', 'b', 'c', 'e')
('a', 'b', 'c', 'f')
('a', 'b', 'c', 'g')
('a', 'b', 'd', 'e')
('a', 'b', 'd', 'f')
('a', 'b', 'd', 'g')
('a', 'b', 'e', 'f')
('a', 'b', 'e', 'g')
('a', 'b', 'f', 'g')
('a', 'c', 'd', 'e')
('a', 'c', 'd', 'f')
('a', 'c', 'd', 'g')
('a', 'c', 'e', 'f')
('a', 'c', 'e', 'g')
('a', 'c', 'f', 'g')
('a', 'd', 'e', 'f')
('a', 'd', 'e', 'g')
('a', 'd', 'f', 'g')
('a', 'e', 'f', 'g')
('b', 'c', 'd', 'e')
('b', 'c', 'd', 'f')
('b', 'c', 'd', 'g')
('b', 'c', 'e', 'f')
('b', 'c', 'e', 'g')
('b', 'c', 'f', 'g')
('b', 'd', 'e', 'f')
('b', 'd', 'e', 'g')
('b', 'd', 'f', 'g')
('b', 'e', 'f', 'g')
('c', 'd', 'e', 'f')
('c', 'd', 'e', 'g')
('c', 'd', 'f', 'g')
('c', 'e', 'f', 'g')
('d', 'e', 'f', 'g')


In [118]:
def run_specific_cfa3(mydata, whichcolumn, whichscale, whichcfa, howmanyitems):
    print('THIS ONE RUNS ALL COMBINATIONS OF ITEMS')
    assert(whichcolumn in ['valid_enriched', 'valid_genpop', 'enriched_withrecontact', 'genpop_withrecontact'])
    assert(whichscale in ['anhedonic_depression', 'anxious_worry', 'appetite_gain', 'hyposomnia', 'insomnia', 'panic', 'separation_insecurity',
                         'shame_guilt', 'situational_phobia', 'social_anxiety', 'well_being'])
    num_iter = 1000
    rprint = ro.r('print')
    summary = ro.r('summary')
    ro.r('set.seed(12345)')
    rngkind = "L'Ecuyer-CMRG"
    mdl_strs = {}
    for ss, df in item_lookup.groupby('subscale'):
        cln_ss = ss.replace(" ", "_").replace("/", "_").replace("-", "_")
        items = []
        for itemn, item in enumerate(df.sort_values('id_ht').htid.values):
            items.append(item)
        mdl_strs[cln_ss] = f'{cln_ss} =~' + ' + '.join(items) 
    # measures we can't work with because too few degrees of freedom:
    # - 'appetite_loss': 'appetite_loss =~hitop109 + hitop280 + hitop283'
    # - 'cognitive_problems': 'cognitive_problems =~hitop142 + hitop159 + hitop189',
    # - 'indecisiveness': 'indecisiveness =~hitop21 + hitop90 + hitop95'
    # here's the dictionary with items that *will* work
    mdl_strs_new = {'anhedonic_depression': 'anhedonic_depression =~hitop39 + hitop77 + hitop84 + hitop92 + hitop93 + hitop123 + hitop157 + hitop182 + hitop230 + hitop246',
     'anxious_worry': 'anxious_worry =~hitop20 + hitop34 + hitop89 + hitop203 + hitop248 + hitop265',
     'appetite_gain': 'appetite_gain =~hitop120 + hitop141 + hitop243 + hitop275',
     'hyposomnia': 'hyposomnia =~hitop99 + hitop181 + hitop5 + hitop66 + hitop231',
     'insomnia': 'insomnia =~hitop160 + hitop254 + hitop261 + hitop268',
     'panic': 'panic =~hitop15 + hitop104 + hitop126 + hitop211 + hitop215 + hitop257',
     'separation_insecurity': 'separation_insecurity =~hitop40 + hitop50 + hitop69 + hitop81 + hitop113 + hitop136 + hitop151 + hitop197',
     'shame_guilt': 'shame_guilt =~hitop72 + hitop140 + hitop143 + hitop220',
     'situational_phobia': 'situational_phobia =~hitop16 + hitop165 + hitop225 + hitop247 + hitop278',
     'social_anxiety': 'social_anxiety =~hitop1 + hitop17 + hitop114 + hitop117 + hitop124 + hitop129 + hitop204 + hitop222 + hitop236 + hitop258',
     'well_being': 'well_being =~hitop9 + hitop23 + hitop54 + hitop106 + hitop149 + hitop200 + hitop244 + hitop245 + hitop250 + hitop281'}
    # to load the data in R, let's save it as csv and then load it to R
    # there may be a more elegant day to do this, but I don't know it...
    if whichcolumn == 'valid_genpop':
        mydata.to_csv('../data/CFA/mydata_valid_genpop.csv')
        ro.r("rdata <- read.csv('../data/CFA/mydata_valid_genpop.csv', header=TRUE)")
        group = 'whichdata'
    elif whichcolumn == 'valid_enriched':
        mydata.to_csv('../data/CFA/mydata_valid_enriched.csv')
        ro.r("rdata <- read.csv('../data/CFA/mydata_valid_enriched.csv', header=TRUE)") 
        group = 'whichdata'
    elif whichcolumn == 'enriched_withrecontact':
        mydata.to_csv('../data/CFA/data_enriched.csv')
        ro.r("rdata <- read.csv('../data/CFA/data_enriched.csv', header=TRUE)") 
        group = 'whichvisit'
    elif whichcolumn == 'genpop_withrecontact':
        mydata.to_csv('../data/CFA/data_genpop.csv')
        ro.r("rdata <- read.csv('../data/CFA/data_genpop.csv', header=TRUE)")
        group = 'whichvisit'
    ro.globalenv['group'] = group
    for cln_ss, mdl in mdl_strs_new.items():
        if cln_ss == whichscale:  
            print('lets go')
            if whichcfa == 'metric':
                do_metric = True
                do_scalar = False
                do_strict = False
            elif whichcfa == 'scalar':
                do_metric = True
                do_scalar = True
                do_strict = False   
            elif whichcfa == 'strict':
                do_metric = True
                do_scalar = True
                do_strict = True                  
            print("\n")
            print(cln_ss.upper())
            print(mdl)
            # ====== CONFIG =====        

            # loop to do ablations
            temp_items = mdl_strs_new[cln_ss]
            temp_items_items = temp_items.split("=~",1)[1]
            temp_items_items_items = temp_items_items.split(" + ")
            list_of_items = temp_items_items_items

            for com in combinations(list_of_items, howmanyitems):
                print('\nCOMBINATIONS OF ITEMS: ')
                print(com)
                mynewstring = temp_items.split("=~",1)[0] + "=~"
                for x in com:
                    mynewstring += x + ' + '
                mynewstring = mynewstring[:-3]
                print(mynewstring)
            
                ro.globalenv[cln_ss+'_mdl'] = mynewstring           
            
                fit_config = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV")')
                ro.globalenv['fit_config'] = fit_config        
                out_config = semtools.permuteMeasEq(nPermute=num_iter, # 100 for now, 1000 for the "real" run
                                                        con=fit_config, # In the case of testing configural invariance when modelType = "mgcfa", con is the configural model (implicitly, the unconstrained model is the saturated model, so use the defaults uncon = NULL and param = NULL).
                                                        parallelType="multicore", ncpus=4)
                config_p = extract_p(out_config)
                if float(config_p) >= 0.05:
                    print('CONFIG INVARIANT chisq p = ' + str(config_p))
                else:
                    (passed_flag, my_cfi, my_tli, my_rmsea) = check_secondary_criteria(fit_config)
                    if passed_flag:
                        print('CONFIG chisq p = ' + str(config_p))
                        print('PASSED SECONDARY CRITERIA WITH CFI = ' + str(my_cfi) + ' TLI = ' + str(my_tli) + ' RMSEA = ' + str(my_rmsea))
                    else:
                        print('CONFIG INVARIANT NOT PASSED, chisq p = ' + str(config_p))
                if do_metric:
                    # ====== METRIC =====
                    fit_metric = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal="loadings")')
                    ro.globalenv['fit_metric'] = fit_metric
                    out_metric = semtools.permuteMeasEq(nPermute=num_iter, 
                                                        uncon=fit_config, 
                                                        con=fit_metric, 
                                                        param="loadings",
                                                        parallelType="multicore", ncpus=4)
                    metric_p = extract_p(out_metric)
                    if float(metric_p) >= 0.05:
                        print('METRIC INVARIANT chisq p = ' + str(metric_p))
                    else:
                        # significant:
                        print('METRIC INVARIANT NOT PASSED, chisq p = ' + str(metric_p))
                        ro.globalenv['out_metric'] = out_metric
                        ro.r("myoutputmetric <- capture.output(summary(out_metric))") 
                        ro.r("stringid <- grep(\"chisq\", myoutputmetric)")
                        ro.r("print(myoutputmetric[stringid-1])")
                        ro.r("print(myoutputmetric[stringid])")
                        # significant:
                        ro.r("stringidsig <- grep(\"may differ between Groups\", myoutputmetric)")
                        ro.r("print(myoutputmetric[stringidsig])")
                    if do_scalar:
                        # ====== SCALAR =====
                        ro.globalenv['out_metric'] = out_metric
                        fit_scalar = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal=c("loadings", "intercepts"))')
                        ro.globalenv['fit_scalar'] = fit_scalar
                        out_scalar = semtools.permuteMeasEq(nPermute=num_iter, 
                                                        uncon=fit_metric, 
                                                        con=fit_scalar, 
                                                        param=ro.StrVector(["loadings","intercepts"]),
                                                        parallelType="multicore", 
                                                        ncpus=4)
                        scalar_p = extract_p(out_scalar)
                        if float(scalar_p) >= 0.05:
                            print('SCALAR INVARIANT chisq p = ' + str(scalar_p))
                        else:
                            # significant:
                            print('SCALAR INVARIANT NOT PASSED, chisq p = ' + str(scalar_p))
                            _ = print_problematic_items(out_scalar)
                            ro.globalenv['out_scalar'] = out_scalar
                            ro.r("myoutputscalar <- capture.output(summary(out_scalar))") 
                            ro.r("stringid <- grep(\"chisq\", myoutputscalar)")
                            ro.r("print(myoutputscalar[stringid-1])")
                            ro.r("print(myoutputscalar[stringid])")
                            # significant:
                            ro.r("stringidsig <- grep(\"may differ between Groups\", myoutputscalar)")
                            ro.r("print(myoutputscalar[stringidsig])")
                        if do_strict:
                            # ====== STRICT =====
                            ro.globalenv['out_scalar'] = out_scalar
                            fit_strict = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal=c("loadings", "intercepts", "residuals"))')
                            ro.globalenv['fit_strict'] = fit_strict
                            out_strict = semtools.permuteMeasEq(nPermute=num_iter, 
                                                                uncon=fit_scalar, 
                                                                con=fit_strict, 
                                                                param=ro.StrVector(["loadings","intercepts", "residuals"]),
                                                                parallelType="multicore", 
                                                                ncpus=4)
                            strict_p = extract_p(out_strict)
                            if float(strict_p) >= 0.05:
                                print('STRICT INVARIANT chisq p = ' + str(strict_p))
                            else:
                                # significant:
                                print('STRICT INVARIANT NOT PASSED, chisq p = ' + str(strict_p))
                                _ = print_problematic_items(out_strict)
                                ro.globalenv['out_strict'] = out_strict
                                ro.r("myoutputstrict <- capture.output(summary(out_strict))") 
                                ro.r("stringid <- grep(\"chisq\", myoutputstrict)")
                                ro.r("print(myoutputstrict[stringid-1])")
                                ro.r("print(myoutputstrict[stringid])")
                                # significant:
                                ro.r("stringidsig <- grep(\"may differ between Groups\", myoutputstrict)")
                                ro.r("print(myoutputstrict[stringidsig])")
                        else:
                            print('STRICT N/A')
                            strict_p = 'NA'
                    else:
                        print('SCALAR N/A')
                        print('STRICT N/A')
                        scalar_p = 'NA'
                        strict_p = 'NA'
                else:
                    print('METRIC N/A')
                    print('SCALAR N/A')
                    print('STRICT N/A')
                    metric_p = 'NA'
                    scalar_p = 'NA'
                    strict_p = 'NA'
    print("DONE")
    return(0)

In [89]:
# experimenting; to delete later

mdl_strs_new = {'anhedonic_depression': 'anhedonic_depression =~hitop39 + hitop77 + hitop84 + hitop92 + hitop93 + hitop123 + hitop157 + hitop182 + hitop230 + hitop246',
     'anxious_worry': 'anxious_worry =~hitop20 + hitop34 + hitop89 + hitop203 + hitop248 + hitop265',
     'appetite_gain': 'appetite_gain =~hitop120 + hitop141 + hitop243 + hitop275',
     'hyposomnia': 'hyposomnia =~hitop99 + hitop181 + hitop5 + hitop66 + hitop231',
     'insomnia': 'insomnia =~hitop160 + hitop254 + hitop261 + hitop268',
     'panic': 'panic =~hitop15 + hitop104 + hitop126 + hitop211 + hitop215 + hitop257',
     'separation_insecurity': 'separation_insecurity =~hitop40 + hitop50 + hitop69 + hitop81 + hitop113 + hitop136 + hitop151 + hitop197',
     'shame_guilt': 'shame_guilt =~hitop72 + hitop140 + hitop143 + hitop220',
     'situational_phobia': 'situational_phobia =~hitop16 + hitop165 + hitop225 + hitop247 + hitop278',
     'social_anxiety': 'social_anxiety =~hitop1 + hitop17 + hitop114 + hitop117 + hitop124 + hitop129 + hitop204 + hitop222 + hitop236 + hitop258',
     'well_being': 'well_being =~hitop9 + hitop23 + hitop54 + hitop106 + hitop149 + hitop200 + hitop244 + hitop245 + hitop250 + hitop281'}
cln_ss = 'anhedonic_depression'

temp_items = mdl_strs_new[cln_ss]
temp_items_items = temp_items.split("=~",1)[1]
temp_items_items_items = temp_items_items.split(" + ")
list_of_items = temp_items_items_items

for item in list_of_items:
    print('Item to remove: ' + item)
    mynewstring = temp_items.split("=~",1)[0] + "=~"
    temp_items_items_items.remove(item)
    for x in temp_items_items_items:
        mynewstring += x + ' + '
    temp_items_items_items = temp_items_items.split(" + ")
    mynewstring = mynewstring[:-3]
    #print(mynewstring)
    #mynewstring = temp_items.split("=~",1)[0] + "=~"
    print(mynewstring)

    ro.globalenv[cln_ss+'_mdl'] = mynewstring           
            
    fit_config = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV")')
    ro.globalenv['fit_config'] = fit_config        
    out_config = semtools.permuteMeasEq(nPermute=1000, # 100 for now, 1000 for the "real" run
                                                        con=fit_config, # In the case of testing configural invariance when modelType = "mgcfa", con is the configural model (implicitly, the unconstrained model is the saturated model, so use the defaults uncon = NULL and param = NULL).
                                                        parallelType="multicore", ncpus=4)
    config_p = extract_p(out_config)
    print(config_p)

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




Item to remove: hitop39
anhedonic_depression =~hitop77 + hitop84 + hitop92 + hitop93 + hitop123 + hitop157 + hitop182 + hitop230 + hitop246


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




0.742
Item to remove: hitop84
anhedonic_depression =~hitop39 + hitop77 + hitop92 + hitop93 + hitop123 + hitop157 + hitop182 + hitop230 + hitop246


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




...

0.198
Item to remove: hitop182
anhedonic_depression =~hitop39 + hitop77 + hitop84 + hitop92 + hitop93 + hitop123 + hitop157 + hitop230 + hitop246


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




0.765
Item to remove: hitop230
anhedonic_depression =~hitop39 + hitop77 + hitop84 + hitop92 + hitop93 + hitop123 + hitop157 + hitop182 + hitop246


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




0.888
Item to remove: hitop246
anhedonic_depression =~hitop39 + hitop77 + hitop84 + hitop92 + hitop93 + hitop123 + hitop157 + hitop182 + hitop230
0.876


In [5]:
item_lookup = load_item_lookup()
item_lookup
#global item_lookup

/Users/zeleninam2/Documents/projects/hitop/2-analysis-CFA/env/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:81: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


,id_cm,subscale,item,id_ht,htid
0,3.0,anhedonic depression,I had very little energy.,157,hitop157
1,9.0,separation insecurity,I worried that others would abandon me.,81,hitop81
2,16.0,anxious worry,Thoughts were racing through my head.,34,hitop34
3,19.0,well-being,It was easy for me to laugh.,54,hitop54
4,23.0,appetite gain,I stuffed myself with food.,243,hitop243
5,27.0,anhedonic depression,I was unable to enjoy things like I normally do.,182,hitop182
6,28.0,separation insecurity,I wanted other people to take care of me.,69,hitop69
7,41.0,anxious worry,I had a lot of nervous energy.,89,hitop89
8,43.0,separation insecurity,I wanted someone else to make decisions for me.,50,hitop50
9,47.0,social anxiety,I avoided performing or giving a talk in front of others.,129,hitop129


In [6]:
data_val = load_valsample()
data_genpop, data_genpop_norecontact = load_data('genpop', doing_checks = True, doing_remove_checks=True)
data_genpop.shape

True
checks_initial:
passed_checks    0.462
passed_grid      0.006
passed_list      0.458
dtype: float64
Passed checks: 269 out of 500
checks_recontact:
passed_checks_recontact    0.548
passed_grid_recontact      0.200
passed_list_recontact      0.548
dtype: float64
Passed checks: 226 out of 500
removing checks
(162, 734)
done removing checks


(162, 167)

In [7]:
mydata_val_genpop = pd.concat([data_val, data_genpop_norecontact])
mydata_val_genpop.shape

(658, 84)

In [8]:
data_enriched, data_enriched_norecontact = load_data('enriched', doing_checks = True, doing_remove_checks=True)
print(data_enriched.shape)
mydata_val_enriched = pd.concat([data_val, data_enriched_norecontact])
print(mydata_val_enriched.shape)

True
checks_initial:
passed_checks    0.293548
passed_grid      0.016129
passed_list      0.290323
dtype: float64
Passed checks: 219 out of 310
checks_recontact:
passed_checks_recontact    0.412903
passed_grid_recontact      0.174194
passed_list_recontact      0.412903
dtype: float64
Passed checks: 182 out of 310
removing checks
(157, 734)
done removing checks
(157, 167)
(653, 84)


In [9]:
data_genpop.head()

,hitop157,hitop81,hitop34,hitop54,hitop243,hitop182,hitop69,hitop89,hitop50,hitop129,hitop265,hitop124,hitop231,hitop93,hitop67,hitop245,hitop281,hitop141,hitop40,hitop204,hitop21,hitop236,hitop280,hitop84,hitop120,hitop77,hitop92,hitop258,hitop39,hitop254,hitop215,hitop95,hitop106,hitop283,hitop16,hitop20,hitop189,hitop1,hitop136,hitop246,hitop248,hitop257,hitop114,hitop117,hitop250,hitop200,hitop160,hitop23,hitop165,hitop244,hitop9,hitop142,hitop230,hitop149,hitop247,hitop99,hitop66,hitop240,hitop222,hitop90,hitop113,hitop278,hitop203,hitop159,hitop123,hitop275,hitop268,hitop225,hitop143,hitop151,hitop181,hitop211,hitop17,hitop126,hitop5,hitop261,hitop220,hitop15,hitop72,hitop140,hitop109,hitop197,hitop104,hitop157_recontact,hitop81_recontact,hitop34_recontact,hitop54_recontact,hitop243_recontact,hitop182_recontact,hitop69_recontact,hitop89_recontact,hitop50_recontact,hitop129_recontact,hitop265_recontact,hitop124_recontact,hitop231_recontact,hitop93_recontact,hitop67_recontact,hitop245_recontact,hitop281_recontact,hitop141_recontact,hitop40_recontact,hitop204_recontact,hitop21_recontact,hitop236_recontact,hitop280_recontact,hitop84_recontact,hitop120_recontact,hitop77_recontact,hitop92_recontact,hitop258_recontact,hitop39_recontact,hitop254_recontact,hitop215_recontact,hitop95_recontact,hitop106_recontact,hitop283_recontact,hitop16_recontact,hitop20_recontact,hitop189_recontact,hitop1_recontact,hitop136_recontact,hitop246_recontact,hitop248_recontact,hitop257_recontact,hitop114_recontact,hitop117_recontact,hitop250_recontact,hitop200_recontact,hitop160_recontact,hitop23_recontact,hitop165_recontact,hitop244_recontact,hitop9_recontact,hitop142_recontact,hitop230_recontact,hitop149_recontact,hitop247_recontact,hitop99_recontact,hitop66_recontact,hitop240_recontact,hitop222_recontact,hitop90_recontact,hitop113_recontact,hitop278_recontact,hitop203_recontact,hitop159_recontact,hitop123_recontact,hitop275_recontact,hitop268_recontact,hitop225_recontact,hitop143_recontact,hitop151_recontact,hitop181_recontact,hitop211_recontact,hitop17_recontact,hitop126_recontact,hitop5_recontact,hitop261_recontact,hitop220_recontact,hitop15_recontact,hitop72_recontact,hitop140_recontact,hitop109_recontact,hitop197_recontact,hitop104_recontact,whichdata
1,2,1,4,4,1,3,1,3,1,2,4,3,1,2,3,4,4,4,2,4,2,3,2,1,2,4,2,2,1,2,1,2,4,1,3,4,2,3,1,2,2,4,2,2,1,4,3,4,4,4,4,2,2,4,4,2,3,2,4,2,2,2,1,3,1,2,1,2,2,2,2,4,2,3,2,2,2,3,1,2,2,1,3,2,1,3,4,1,2,1,2,1,3,4,2,2,2,1,4,4,4,1,3,3,2,2,1,1,4,2,2,2,4,1,2,4,3,4,2,3,2,1,2,2,2,1,2,2,2,3,4,4,4,4,2,2,4,4,3,2,2,4,2,1,1,1,2,1,2,2,2,1,1,2,2,2,2,2,3,1,2,1,1,2,1,2,genpop
4,4,1,2,4,4,4,4,1,2,3,1,2,1,3,3,2,1,4,3,2,2,3,1,2,4,3,2,2,4,4,2,2,1,1,1,2,3,2,1,2,3,4,2,2,1,3,4,2,1,1,2,4,1,1,2,3,1,1,1,2,1,1,3,3,1,4,2,2,2,3,1,2,2,2,1,4,2,2,2,2,1,1,3,2,1,2,1,3,3,2,1,2,1,2,1,1,3,3,1,2,1,2,2,2,2,2,3,3,3,2,2,3,3,1,3,1,2,1,3,3,2,1,2,3,2,3,2,3,1,3,2,1,1,1,3,2,1,1,2,2,3,1,2,1,1,3,3,2,4,2,1,2,2,1,2,1,1,1,3,2,2,2,2,3,1,3,genpop
6,4,1,2,3,1,1,1,1,1,1,1,1,1,1,2,3,3,1,1,1,2,1,2,2,1,2,3,1,1,3,1,2,1,1,1,2,2,1,1,2,1,1,1,1,1,2,4,3,1,2,2,2,1,2,1,2,1,1,1,2,1,1,2,2,1,3,3,2,1,2,1,1,1,1,1,3,2,1,1,2,2,1,1,3,1,2,1,1,2,1,1,2,1,1,1,1,1,1,3,3,1,1,1,2,1,2,2,1,1,3,1,2,3,1,2,2,1,1,1,1,1,1,1,1,1,1,1,2,2,4,3,1,2,2,2,1,2,1,1,1,1,1,2,1,1,2,2,1,2,2,1,1,1,1,1,1,1,1,3,2,1,1,1,2,1,1,genpop
10,2,1,1,3,1,1,3,1,2,1,1,1,1,1,2,2,2,1,1,1,2,2,1,1,1,2,1,1,1,2,1,2,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,2,1,2,3,1,1,1,1,1,1,1,1,2,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,2,1,2,3,1,1,2,2,1,1,1,1,1,1,2,4,3,1,1,1,2,2,1,2,1,2,1,1,1,2,1,2,1,1,1,2,2,2,1,1,1,2,1,1,2,1,1,3,1,2,3,2,2,2,1,1,1,1,1,2,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,genpop
12,4,1,4,1,1,3,2,3,2,4,3,3,3,3,3,2,1,1,1,4,4,4,4,4,1,4,4,4,4,4,3,2,1,4,1,3,4,4,2,4,1,4,4,4,1,3,3,1,1,1,1,4,2,1,2,4,4,4,3,3,4,4,4,3,4,1,4,1,4,4,3,2,3,2,3,3,2,2,2,2,4,4,2,3,2,2,1,1,3,1,2,2,4,2,4,3,4,4,2,1,1,2,4,3,3,3,3,1,3,3,4,3,4,2,3,1,2,1,3,3,3,3,3,1,2,3,3,1,3,2,1,1,1,1,3,3,1,2,3,3,3,2,2,4,3,3,3,4,1,4,1,3,3,3,2,3,3,3,3,3,2,3,4,4,3,3,genpop


In [10]:
data_genpop_recontactanalysis = prep_data_recontactanalysis(data_genpop)
data_enriched_recontactanalysis = prep_data_recontactanalysis(data_enriched)

In [108]:
# data that exists:
# - mydata_val_genpop
# - mydata_val_enriched
# - data_genpop_recontactanalysis
# - data_enriched_recontactanalysis
my_df = do_cfa(mydata = data_enriched_recontactanalysis, whichcolumn = 'enriched_withrecontact') 
# whichcolumn == 'valid_genpop', 'valid_enriched', 'enriched_withrecontact', 'genpop_withrecontact':

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.







ANHEDONIC_DEPRESSION
anhedonic_depression =~hitop39 + hitop77 + hitop84 + hitop92 + hitop93 + hitop123 + hitop157 + hitop182 + hitop230 + hitop246
----------------


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




CONFIG INVARIANT chisq p = 0.491


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




...

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




SCALAR N/A
STRICT N/A



APPETITE_GAIN
appetite_gain =~hitop120 + hitop141 + hitop243 + hitop275
----------------


R[write to console]: 

R[write to console]: There were 50 or more warnings (use warnings() to see the first 50)
R[write to console]: 



CONFIG INVARIANT chisq p = 0.947


RRuntimeError: Error in lav_data_full(data = data, group = group, cluster = cluster,  : 
  lavaan ERROR: grouping variable ‘whichvisit’ not found;
  variable names found in data frame are:
  X hitop157 hitop81 hitop34 hitop54 hitop243 hitop182 hitop69 hitop89 hitop50 hitop129 hitop265 hitop124 hitop231 hitop93 hitop67 hitop245 hitop281 hitop141 hitop40 hitop204 hitop21 hitop236 hitop280 hitop84 hitop120 hitop77 hitop92 hitop258 hitop39 hitop254 hitop215 hitop95 hitop106 hitop283 hitop16 hitop20 hitop189 hitop1 hitop136 hitop246 hitop248 hitop257 hitop114 hitop117 hitop250 hitop200 hitop160 hitop23 hitop165 hitop244 hitop9 hitop142 hitop230 hitop149 hitop247 hitop99 hitop66 hitop240 hitop222 hitop90 hitop113 hitop278 hitop203 hitop159 hitop123 hitop275 hitop268 hitop225 hitop143 hitop151 hitop181 hitop211 hitop17 hitop126 hitop5 hitop261 hitop220 hitop15 hitop72 hitop140 hitop109 hitop197 hitop104 hitop157_recontact hitop81_recontact hitop34_recontact hitop54_recontact hitop243_recontac

# --- Ablation experiments ---

In [98]:
run_specific_cfa(data_enriched_recontactanalysis, 
                 whichcolumn = 'enriched_withrecontact', 
                 whichscale = 'anxious_worry', 
                 whichcfa = 'metric')

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




! DONT FORGET TO CHANGE MDL_STRS_NEW BELOW IN CODE FOR ABLATION EXPERIMENTS
lets go


ANXIOUS_WORRY
anxious_worry =~hitop20 + hitop34 + hitop89 + hitop203 + hitop248 + hitop265


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




CONFIG INVARIANT chisq p = 0.957
METRIC INVARIANT NOT PASSED, chisq p = 0.004
[1] "      AFI.Difference p.value"
[1] "chisq         23.399   0.004"
character(0)
SCALAR N/A
STRICT N/A
DONE


0

In [133]:
run_specific_cfa2(mydata_val_genpop, 
                 whichcolumn = 'valid_genpop', 
                 whichscale = 'well_being', 
                 whichcfa = 'strict')

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




! DONT FORGET TO CHANGE MDL_STRS_NEW BELOW IN CODE FOR ABLATION EXPERIMENTS
lets go


WELL_BEING
well_being =~hitop9 + hitop23 + hitop54 + hitop106 + hitop149 + hitop200 + hitop244 + hitop245 + hitop250 + hitop281

ITEM TO REMOVE: hitop9
well_being =~hitop23 + hitop54 + hitop106 + hitop149 + hitop200 + hitop244 + hitop245 + hitop250 + hitop281


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




CONFIG chisq p = 0.011
PASSED SECONDARY CRITERIA WITH CFI = 1.0 TLI = 1.004 RMSEA = 0.035
METRIC INVARIANT NOT PASSED, chisq p = 0.003
[1] "      AFI.Difference p.value"
[1] "chisq         42.304   0.003"
[1] "Parameter 'well_being=~hitop54' may differ between Groups 'validation' and 'genpop'."


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




...

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




SCALAR INVARIANT NOT PASSED, chisq p = 0
Problematic items:
[1] "Parameter 'well_being=~hitop54' may differ between Groups 'validation' and 'genpop'."
[1] "      AFI.Difference p.value"
[1] "chisq         33.804       0"
[1] "Parameter 'well_being=~hitop54' may differ between Groups 'validation' and 'genpop'."


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




STRICT INVARIANT chisq p = 0.348
DONE


0

In [135]:
# let's do stubborn ones
run_specific_cfa3(mydata_val_genpop, 
                 whichcolumn = 'valid_genpop', 
                 whichscale = 'well_being', 
                 whichcfa = 'metric',
                 howmanyitems = 8)

THIS ONE RUNS ALL COMBINATIONS OF ITEMS
lets go


WELL_BEING
well_being =~hitop9 + hitop23 + hitop54 + hitop106 + hitop149 + hitop200 + hitop244 + hitop245 + hitop250 + hitop281

COMBINATIONS OF ITEMS: 
('hitop9', 'hitop23', 'hitop54', 'hitop106', 'hitop149', 'hitop200', 'hitop244', 'hitop245')
well_being =~hitop9 + hitop23 + hitop54 + hitop106 + hitop149 + hitop200 + hitop244 + hitop245


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




CONFIG INVARIANT chisq p = 0.146
METRIC INVARIANT NOT PASSED, chisq p = 0
[1] "      AFI.Difference p.value"
[1] "chisq         40.991       0"
[1] "Parameter 'well_being=~hitop54' may differ between Groups 'validation' and 'genpop'."


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




SCALAR N/A
STRICT N/A

COMBINATIONS OF ITEMS: 
('hitop9', 'hitop23', 'hitop54', 'hitop106', 'hitop149', 'hitop200', 'hitop244', 'hitop250')
well_being =~hitop9 + hitop23 + hitop54 + hitop106 + hitop149 + hitop200 + hitop244 + hitop250


...

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




SCALAR N/A
STRICT N/A

COMBINATIONS OF ITEMS: 
('hitop54', 'hitop106', 'hitop149', 'hitop200', 'hitop244', 'hitop245', 'hitop250', 'hitop281')
well_being =~hitop54 + hitop106 + hitop149 + hitop200 + hitop244 + hitop245 + hitop250 + hitop281


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




CONFIG chisq p = 0.005
PASSED SECONDARY CRITERIA WITH CFI = 0.997 TLI = 0.995 RMSEA = 0.035
METRIC INVARIANT NOT PASSED, chisq p = 0.036
[1] "      AFI.Difference p.value"
[1] "chisq         24.636   0.036"
character(0)
SCALAR N/A
STRICT N/A
DONE


0

In [134]:
items_to_check = ['9', '23', '54', '106', '149', '200', '244', '245', '250', '281']
for i in items_to_check:
    a = repr(item_lookup.loc[item_lookup['htid'] == 'hitop'+i]['htid']).split(' ', 1)[1]
    aa = str(a).split('\n')[0]
    b = repr(item_lookup.loc[item_lookup['htid'] == 'hitop'+i]['item']).split(' ', 1)[1]
    bb = str(b).split('\n')[0]
    print(aa+' : '+bb)

   hitop9 :    I felt like I was having a lot of fun.
   hitop23 :    I felt cheerful.
   hitop54 :    It was easy for me to laugh.
   hitop106 :    I was proud of myself.
   hitop149 :    I felt good about myself.
   hitop200 :    I found compliments very encouraging.
   hitop244 :    I felt optimistic.
   hitop245 :    I looked forward to things with enjoyment.
   hitop250 :    I felt like I had a lot of interesting things to do.
   hitop281 :    I felt that I had a lot to look forward to.
